In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.agent import AgentFinish
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from pydantic import BaseModel, Field
from llama_index.core import SimpleDirectoryReader
from pydantic import BaseModel, Field

In [2]:
openai_llm = ChatOpenAI(temperature=0, model="gpt-4o")

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_10584\1305393998.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  openai_llm = ChatOpenAI(temperature=0, model="gpt-4o")


In [3]:
cv_template ="""

You are an expert in analyzing CVs and providing personalized career advice. Given an individual's CV, you will:

1. Conduct a detailed analysis of their skills, experience, qualifications, and overall suitability for various roles.
2. Recommend the best job opportunities that align with their profile, ensuring that the suggestions are well-matched to their strengths and career goals.
3. Take into account current industry trends, job market demand, and emerging opportunities to ensure that your recommendations are both relevant and forward-thinking.
4. Provide actionable insights that can help improve the CV, such as highlighting areas for growth or skill enhancement based on market needs.

Your goal is to give comprehensive, data-driven advice that helps individuals make informed career decisions.

below is the CV:
{input}


The output should been a python dict have the following keywords

Skills_and_experience_analysis,
job_opportunities
industry_trends_and_market_demand
cv_improvement_suggestions
"""
email_template = """
You are an expert in writing application letters. Your task is to create a compelling and professional cover letter based on an individual's skills and experience, showcasing how they align with the job they're applying for.

Please ensure that the application letter includes:
- A strong introduction highlighting the candidate’s background and motivation for applying.
- A section detailing the key skills and experience that make the candidate a strong fit for the position.
- A concluding paragraph that emphasizes the candidate’s enthusiasm and readiness for the role.

Use a professional tone, and ensure the letter is clear and focused on presenting the candidate in the best possible light.

The candidate's CV:
{input}

The job being applied for is:
{job_description}

Please format the letter as follows:
1. Introduction
2. Skills and Experience
3. Closing and Enthusiasm
"""




In [4]:
import gradio as gr
import os
def get_file(file_path):
    directory_name = os.path.dirname(file_path)  
    
    documents = SimpleDirectoryReader(directory_name).load_data()    
    prompt = ChatPromptTemplate.from_template(cv_template)
    chain = LLMChain(prompt=prompt, llm=openai_llm)
    global doc_info 
    doc_info = documents[0].text
    global cv_llm_response
    cv_llm_response = chain.run({"input":doc_info})
    return cv_llm_response
demo = gr.Interface(fn=get_file, inputs=gr.File(), outputs="text")


d:\Rhodrick\langChain\Job application assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\Design Dep\AppData\Local\Temp\ipykernel_10584\373388985.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=openai_llm)
C:\Users\Design Dep\AppData\Local\Temp\ipykernel_10584\373388985.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  cv_llm_response = chain.run({"input":doc_info})


In [7]:
demo.close()

Closing server running on port: 7860


In [6]:
import json
job_opportunities = []

cleaned_response_string = cv_llm_response.strip('```python\n').strip('```')
json_response = json.loads(cleaned_response_string)

if json_response['job_opportunities'] is not None:
    for idx,job in enumerate(json_response['job_opportunities']):
        response = search_for_jobs(job)
        print(job)

Software Developer
AI Engineer
Machine Learning Engineer
Web Developer
Technical Trainer


In [ ]:
class CVDetails(BaseModel):
    cv_summary :str = Field(description="The overall summary of the person Cv")

In [76]:
search_for_jobs()

h
h


In [138]:
from bs4 import BeautifulSoup
import requests

def search_for_jobs():
    url = "https://myjobo.com/search-jobs?keywords=developer"
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, "lxml")
    job_results = soup.find_all("div", class_="job-card")
    job_details = []

    if len(job_results) > 0:
        for job in job_results:
            job_div = job.find('div', class_="card-body p-0")
            job_title = ""
            if job_div:
                job_title_tag = job_div.find("h5", class_="card-title text-secondary fs-18 mb-0")
                if job_title_tag:
                    job_title = job_title_tag.text.strip()
            
                job_div_loc = job_div.find_all("div", class_="desc d-flex me-4")
                job_location = ""
                for i in job_div_loc:
                    info = i.find("p", class_="fs-14 text-gray mb-2")
                    if info:
                        job_location = info.text.strip()
                post_date_div = job.find("div", class_="desc d-flex")
                post_date = ""
                if post_date_div:
                    post_date_p = post_date_div.find("p", class_="fs-14 text-gray mb-2")
                    if post_date_p:
                        post_date = post_date_p.text.strip()
                
                job_descprition_link_div = job.find("div", class_="mb-40")
                job_desc_link = ""
                if job_descprition_link_div:
                    job_desc_link_a = job_descprition_link_div.find("a", class_="card py-30 border-0")
                    if job_desc_link_a and 'href' in job_desc_link_a.attrs:
                        job_desc_link = job_desc_link_a['href'].strip()
                
               
                job_description_html = requests.get(job_desc_link).text
                desc_soup = BeautifulSoup(job_description_html, "lxml")
                
                
                job_description_div = desc_soup.find("div", class_="job-card")
                responsibilities_div = job_description_div.find("div", class_="key-responsibilities")
                responsibilities_list = []
                if responsibilities_div:
                    responsibilities = responsibilities_div.find_all("p")
                    for resp in responsibilities:
                        responsibilities_list.append(resp.text.strip())
                
                job_details.append({
                    'job_title': job_title,
                    'job_location': job_location,
                    'post_date': post_date,
                    'job_desc_link': job_desc_link,
                    'responsibilities': responsibilities_list
                })
                
    return job_details

In [ ]:
search_for_jobs()